In [ ]:
from blackjacksim.entities import *
from blackjacksim.strategies import basic
from tqdm import tnrange
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
# import jtplot module in notebook
from jupyterthemes import jtplot
from multiprocessing import Process, Manager, Pool

# set "context" (paper, notebook, talk, poster)
# scale font-size of ticklabels, legend, etc.
# remove spines from x and y axes and make grid dashed
jtplot.style(context='poster', fscale=1.4, spines=False, gridlines='--')

startingPool = 3000
    
manager = Manager()
data = manager.list()
def one_round(x):
    house = Blackjack32()
    wallet = PlayerWallet(1,startingPool)
    d = Dealer()
    p = Player(basic, wallet, house)
    s = Shoe(8,.25)
    for t in range(100):
        s = p.deal(s)
        s = d.deal(s)
        s = p.play(s, d.up_card)
        s = d.play(s, p.hands)
        for h in p.hands:
            wallet.take_payout(house.payout(h, d.hand))
        rw, rp = wallet.finish_round()
        data.append({'Round':t,'Pool':wallet.wager_pool, 'Wager':rw, 'Payout':rp, 'Shoe Penetration':s._penetration_state})
        
processes = []
with Pool(processes=12) as pool:
    max_ = 10000
    with tqdm(total=max_) as pbar:
        for i, _ in enumerate(pool.imap_unordered(one_round, range(0, max_))):
            pbar.update()
            
df = pd.DataFrame(list(data))
df['Advantage'] = (df.Payout-df.Wager)
ax = sns.lineplot(x='Round', y='Advantage', data=df)
ax.axhline(y=df.Advantage.mean(), color='r')
ax.set_title('Player Advantage {:.3f}%'.format(df.Advantage.mean()*100))
